<div class="header">
  <h1 style="float: left;">Visualizing Temporal Networks</h1>
  <img width=300px src="https://www.uni-koblenz-landau.de/de/koblenz/logo.png" style="position: relative;"/>
</div>

In [ ]:
################
# Dependencies #
################
# Note: Never import anything directly always import modules or packages, not single functions or classes,
# We should also check and clean up dependencies at some point.
import fileupload
import IPython.display as ipydisplay
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import plotly.offline

import traceback

import vtna.data_import
import vtna.graph
import vtna.layout
import vtna.utility

import main

plotly.offline.init_notebook_mode(connected=True)

##################
# CSS/JS Imports #
##################

import_html = ''

# Queries
with open('css/queries.css', mode='rt') as f:
    import_html += f'<style>{f.read()}</style>'
with open('js/queries.js', mode='rt') as f:
    import_html += f'<script>{f.read()}</script>'

# Metadata sorting drag and drop
with open('css/dragndrop.css', mode='rt') as f:
    import_html += f'<style>{f.read()}</style>'
# TODO: Use minimal version of js
with open('js/html.sortable.js', mode='rt') as f:
    import_html += f'<script>{f.read()}</script>'
with open('js/dragndrop.js', mode='rt') as f:
    import_html += f'<script>{f.read()}</script>'

# Exporting functionality
with open('js/export.js', mode='rt') as f:
    import_html += f'<script>{f.read()}</script>'

# Statistics cell
with open('css/statistics.css', mode='rt') as f:
    import_html += f'<style>{f.read()}</style>'

# Tooltips
with open('css/tooltips.css', mode='rt') as f:
    import_html += f'<style>{f.read()}</style>'
with open('js/tooltips.js', mode='rt') as f:
    import_html += f'<script>{f.read()}</script>'
    
ipydisplay.display(ipydisplay.HTML(import_html))

###############
# Data Import #
###############
# TODO: Would be great to have notes on where each layout is used.
box_layout = widgets.Layout(margin="10px 0px 10px 0px")
simbox_layout = widgets.Layout(border='solid 1px rgb(210,210,210)',margin="5px 5px 5px 5px", 
                               width="100%")
upload_layout = widgets.Layout(width="100%")

# Help widgets for upload
graph_upload_help = main.help_widget(main.HELP_TEXT['graph_upload'])
metadata_upload_help = main.help_widget(main.HELP_TEXT['metadata_upload'])

# Toggle button: Switch between Computer and Network upload
upload_type_toggle_button = widgets.ToggleButtons(
    options=['Computer', 'Network'],
    description='Source:',
    disabled=False,
    button_style='info',
    tooltips=[main.TOOLTIP['toggle_local_upload'], main.TOOLTIP['toggle_network_upload']],
    layout=upload_layout
)
# Text input for Graph file path
graph_data_text = widgets.Text(
    value='',
    description='Graph File:',
    disabled=True,
    layout=upload_layout
)
# Text input for Metadata file path
metadata_text = widgets.Text(
    value='',
    description='Metadata File:',
    disabled=True, 
    layout=upload_layout
)

# Local upload for graph data
local_graph_file_upload = fileupload.FileUploadWidget(    
    label="Upload",
    tooltip=main.TOOLTIP['graph_upload_button']
)
# Local upload for metadata
local_metadata_file_upload = fileupload.FileUploadWidget(    
    label="Upload",
    tooltip=main.TOOLTIP['metadata_upload_button']
)
# Network upload for graph data
network_graph_upload_button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info',
    tooltip=main.TOOLTIP['graph_upload_button'],
    icon='upload'
)
network_graph_upload_button.layout.display = 'none'
# Network upload for metadata
network_metadata_upload_button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info', 
    tooltip=main.TOOLTIP['metadata_upload_button'],
    icon='upload'
)
loading_layout = widgets.Layout(
    height='350px',
    width='500px'
)
graph_data_loading = main.LoadingIndicator('small', loading_layout)
metadata_loading = main.LoadingIndicator('small', loading_layout)
graph_data_loading_box = graph_data_loading.get_box()
metadata_loading_box = metadata_loading.get_box()

network_metadata_upload_button.layout.display = 'none'
# Summary/Error output for graph data upload
graph_data_output = widgets.Output()
# Graph interactions histogram output
graph_hist_output = widgets.Output()
# Summary/Error output for metadata upload
metadata_output = widgets.Output()

# The container box that will be filled with measure checkboxes
measures_select_box = widgets.VBox(layout=widgets.Layout(
    margin="0px 0px 20px 10px"
))
# Run button applies changes to metadata and open display view
run_button = widgets.Button(
    description='Display Graph',
    disabled=True,
    button_style='success',
    tooltip='',
    icon='start'
)
# TODO: Remove autostart
# Automatically uploads predefined network datasets and displays graph
autostart_button = widgets.Button(
    description='Autostart',
    disabled=False,
    button_style='warning',
    tooltip='Automatically loads graph and metadata of a sociopatterns network. WARNING: May cause unexpected behaviour.',
    icon='start'
)
# Import menu button is shown in any not Import view and switches back to Import view.
import_menu_button = widgets.Button(
    description='Import',
    disabled=False,
    button_style='warning',
    tooltip=main.TOOLTIP['back_to_import_button'],
    icon='start'
)

ordinal_metadata_help = main.help_widget(main.HELP_TEXT['column_ordinal_config'])

# Box for toggle button
upload_type_hbox = widgets.HBox([upload_type_toggle_button], layout=box_layout)
# Box for graph data import
graph_data_upload_hbox = widgets.HBox([graph_upload_help, graph_data_text, local_graph_file_upload, 
                                       network_graph_upload_button])
graph_data_configuration_vbox = widgets.VBox([])
w_network_settings = widgets.VBox([
    widgets.HBox([graph_data_output, graph_data_loading_box, graph_data_configuration_vbox]),
    graph_hist_output
])
import_graph_data_vbox = widgets.VBox([graph_data_upload_hbox, w_network_settings], layout=box_layout)
# Box for metadata import
metadata_upload_hbox = widgets.HBox([metadata_upload_help, metadata_text, local_metadata_file_upload, 
                                     network_metadata_upload_button])
metadata_configuration_left_vbox = widgets.VBox([], layout=widgets.Layout(margin='0px 10px 0px 0px'))
metadata_configuration_hbox = widgets.HBox([metadata_loading_box, metadata_configuration_left_vbox, 
                                            metadata_output, ordinal_metadata_help])
import_metadata_vbox = widgets.VBox([metadata_upload_hbox, metadata_configuration_hbox], layout=box_layout)
# Box for all import functionality
w_toolbar = widgets.HBox([run_button, autostart_button])

####################
# MAIN IMPORT VIEW #
####################
full_import_vbox = widgets.VBox([upload_type_hbox, import_graph_data_vbox, import_metadata_vbox, measures_select_box, w_toolbar])
   
# Create manager for import functionality
upload_manager = main.UIDataUploadManager(
                     run_button=run_button,
                     local_graph_file_upload=local_graph_file_upload,
                     network_graph_upload_button=network_graph_upload_button,
                     graph_data_text=graph_data_text,
                     graph_data_output=graph_data_output,
                     graph_hist_output=graph_hist_output,
                     graph_data_loading=graph_data_loading,
                     local_metadata_file_upload=local_metadata_file_upload,
                     network_metadata_upload_button=network_metadata_upload_button,
                     metadata_text=metadata_text,
                     metadata_output=metadata_output,
                     metadata_loading=metadata_loading,
                     metadata_configuration_vbox=metadata_configuration_left_vbox,
                     metadata_ordinal_help=ordinal_metadata_help,
                     column_configuration_layout=box_layout,
                     graph_data_configuration_vbox=graph_data_configuration_vbox,
                     measures_select_box=measures_select_box
                )
# Attach event handlers to each interactable import widget
upload_type_toggle_button.observe(upload_manager.build_on_toggle_upload_type(), 'value')
local_graph_file_upload.observe(upload_manager.build_handle_upload_graph_data(main.UIDataUploadManager.UploadOrigin.LOCAL), names='data')
local_metadata_file_upload.observe(upload_manager.build_handle_upload_metadata(main.UIDataUploadManager.UploadOrigin.LOCAL), names='data')
network_graph_upload_button.on_click(upload_manager.build_handle_upload_graph_data(main.UIDataUploadManager.UploadOrigin.NETWORK))
network_metadata_upload_button.on_click(upload_manager.build_handle_upload_metadata(main.UIDataUploadManager.UploadOrigin.NETWORK))

######################
# Attribute Ordering #
######################
# Initialize dict that is used by drag n drop sorting/ordering. Used by Javascript
order_dict = dict()
# Initialize dict that indicates attributes being enabled/disabled. Used by Javascript
order_enabled = dict()

################
# Queries Menu #
################
filter_box_layout = widgets.Layout(overflow_y='scroll',
                                   border='solid 1px rgb(210,210,210)',
                                   height='auto',
                                   display='block')

# Global functions to fill by init_queries_manager, used by Javascript engine
addQueryClause = None
deleteQueryClause = None
deleteQuery = None
paintQuery = None
switchQuery = None

def init_queries_manager(attribute_info, queries_vbox):
    global addQueryClause
    global deleteQueryClause
    global deleteQuery
    global paintQuery
    global switchQuery
    queries_manager = main.UIAttributeQueriesManager(attribute_info, queries_vbox, filter_box_layout, 
                                                     'html/query.fragment.mustache')
    # Fill functions for Javascript engine
    addQueryClause = queries_manager.build_add_query_clause()
    deleteQueryClause = queries_manager.build_delete_query_clause()
    deleteQuery = queries_manager.build_delete_query()
    paintQuery = queries_manager.build_paint_query()
    switchQuery = queries_manager.build_switch_query()
    
    return queries_manager


#################
# Graph Display #
#################
# Contains:
# - Display of graph
# - Layout dropdown selection
# - Queries configuration

display_size = 900,800

module_inner_layout = widgets.Layout(padding='10px')
display_layout = widgets.Layout(
    width=str(display_size[0])+'px', 
    height=str(display_size[1])+'px' 
)
# Loading indicator
loading_graph = main.LoadingIndicator('big', display_layout)

# Graph plot output
display_output = widgets.Output()
display_vbox = widgets.VBox(children=[loading_graph.get_box(), display_output], layout=display_layout)

# Layout selection widgets
layout_module_header_html=widgets.HTML('<h4 class="module-header"><i class="fa fa-cogs" style="color:#3498db" aria-hidden="true"></i> Layout settings</h4>', 
                                        layout=widgets.Layout(margin='0px'))
layout_select_vbox = widgets.VBox(layout=module_inner_layout)
layout_module_vbox = widgets.VBox([layout_module_header_html, layout_select_vbox], 
                                  layout=widgets.Layout(border='solid 1px rgb(210,210,210)', margin='5px', 
                                                      width='40%'))
# Querying widgtes
queries_module_header_html=widgets.HTML('<h4 class="module-header"><i class="fa fa-search" style="color:#2ecc71" aria-hidden="true"></i> Queries </h4>', 
                                        layout=widgets.Layout(margin='0px'))
queries_menu_vbox = widgets.VBox(layout=module_inner_layout)
queries_module_vbox = widgets.VBox(children=[queries_module_header_html, queries_menu_vbox],
                                       layout=widgets.Layout(border='solid 1px rgb(210,210,210)', margin='5px', 
                                                      width='60%'))
# Merge queries and layout widgets
queries_and_layout_merge_hbox = widgets.HBox([queries_module_vbox, layout_module_vbox])

# Style widgets
style_module_header_html=widgets.HTML('<h4 class="module-header"><i class="fa fa-cogs" style="color:#3498db" aria-hidden="true"></i> Style options</h4>', 
                                        layout=widgets.Layout(margin='0px'))
style_options_vbox = widgets.VBox(layout=module_inner_layout)
style_module_vbox = widgets.VBox([style_module_header_html, style_options_vbox], 
                                layout=widgets.Layout(border='solid 1px rgb(210,210,210)', margin='5px', 
                                                      width='40%'))
# Export widgets
export_module_header_html=widgets.HTML('<h4 class="module-header"><i class="fa fa-cogs" style="color:#3498db" aria-hidden="true"></i> Exporting</h4>', 
                                        layout=widgets.Layout(margin='0px'))
export_vbox = widgets.VBox(layout=module_inner_layout)
export_module_vbox = widgets.VBox([export_module_header_html, export_vbox],
                                 layout=widgets.Layout(border='solid 1px rgb(210,210,210)', margin='5px', 
                                                       width='60%'))
# Merge style and export widgets
style_and_export_merge_hbox = widgets.HBox([style_module_vbox, export_module_vbox])

# Cumulative option widget
cumulative_hbox = widgets.HBox()

style_manager = main.UIDefaultStyleOptionsManager(style_options_vbox)
# Create Display manager
display_manager = main.UIGraphDisplayManager(display_output=display_output, 
                                             display_size=display_size,
                                             layout_vbox=layout_select_vbox,
                                             export_vbox=export_vbox,
                                             cumulative_hbox=cumulative_hbox,
                                             loading_indicator=loading_graph,
                                             style_manager=style_manager
                                            )

###################
# Statistics Menu #
###################
node_search_vbox = widgets.VBox()
node_detailed_view_vbox = widgets.VBox()

graph_header_hbox = widgets.HBox()
node_summary_hbox = widgets.HBox()
graph_summary_hbox = widgets.HBox()
node_details_hbox = widgets.HBox([node_search_vbox, node_detailed_view_vbox])

main_stats_hbox = widgets.VBox([graph_summary_hbox,
                                node_summary_hbox,
                                node_details_hbox])

statistics_manager = main.UIStatisticsManager(graph_header_hbox=graph_header_hbox,
                                              graph_summary_hbox=graph_summary_hbox, 
                                              node_summary_hbox=node_summary_hbox,
                                              node_search_vbox=node_search_vbox,
                                              node_detailed_view_vbox=node_detailed_view_vbox,
                                              graph_summary_template_path='html/graph_summary.fragment.mustache',
                                              graph_header_template_path='html/graph_header.fragment.mustache'
                                             )

###########################################
# Menu: Toggle between Import and Display #
###########################################
###################
# MAIN GRAPH VIEW #
###################
simulation_box = widgets.VBox([import_menu_button, graph_header_hbox, display_vbox, cumulative_hbox,
                               queries_and_layout_merge_hbox, style_and_export_merge_hbox], 
                              layout=simbox_layout)
# Hide it initially
simulation_box.layout.display = 'none'
# Display the hidden box
ipydisplay.display(simulation_box)

# Note: We should not build and initialize the simulation box before we have the data.
#     Because without the data a number of functions cannot be properly initiliazed.
#     The global simulation_box is a workaround, it should be replaced by some manager class.
#     It is possible that we should rename the display manager to graph manager,
#     and create a display manager responsible for the simulation box.
def on_run(b):
    # on_run will initialize the simulation_box, which contains the graph display.
    # it will initialize the graph object using the display_manager
    global simulation_box
    global display_manager
    global upload_manager
    global loading_graph
    global queries_manager
    global full_import_vbox
    global statistics_manager
    # Hide import view
    full_import_vbox.layout.display = 'none'
    # Hide plot + all configuration boxes that need the loaded data
    display_output.layout.display = 'none' # The graph plot
    queries_and_layout_merge_hbox.layout.display = 'none' 
    style_and_export_merge_hbox.layout.display = 'none' 
    cumulative_hbox.layout.display = 'none'
    # Show graph display view
    simulation_box.layout.display = 'block'
    
    loading_graph.start()

    # Load imported imported graph data, metadata and queries_manager into the temporal graph
    try:
        display_manager.init_temporal_graph(
            edge_list=upload_manager.get_edge_list(),
            metadata=upload_manager.get_metadata(),
            granularity=upload_manager.get_granularity(),
            selected_measures=upload_manager.get_selected_measures()
        )
        
        # Init queries manager
        queries_manager = init_queries_manager(display_manager.get_temporal_graph(), 
                                               queries_menu_vbox)
        display_manager.init_queries_manager(queries_manager)

        # Load temporal graph into statistics manager
        statistics_manager.load(display_manager.get_temporal_graph())

        display_manager.display_graph()
    except Exception as exception:
        loading_graph.stop()
        with display_output:
            ipydisplay.clear_output()
            if isinstance(exception, vtna.graph.MissingNodesInMetadataError):
                print('\x1b[31mInvalid metadata (not all nodes are described)\x1b[0m')
            else:
                print(f'\x1b[31mError while creating the graph:\x1b[0m')
                traceback.print_exc()
        display_output.layout.display = 'block'
        return

    
    # Show hidden graph plot now that it's loaded
    display_output.layout.display = 'block'
    queries_and_layout_merge_hbox.layout.display = 'flex' 
    style_and_export_merge_hbox.layout.display = 'flex'  
    cumulative_hbox.layout.display = 'flex'
    loading_graph.stop()

    
def on_autorun(b):
    global graph_data_text
    global metadata_text
    global upload_manager
    graph_data_text.value = "http://www.sociopatterns.org/wp-content/uploads/2015/09/primaryschool.csv.gz"
    metadata_text.value = "http://www.sociopatterns.org/wp-content/uploads/2015/09/metadata_primaryschool.txt"
    upload_manager.build_handle_upload_graph_data(main.UIDataUploadManager.UploadOrigin.NETWORK)(None)
    upload_manager.build_handle_upload_metadata(main.UIDataUploadManager.UploadOrigin.NETWORK)(None)
    on_run(None)
    
def on_import(b):
    """
    on_import resets and hides the graph display and shows the data import view.
    """
    global simulation_box
    global display_manager
    global full_import_vbox
    # Hide graph view
    simulation_box.layout.display = 'none'
    # Reset graph display manager
    display_manager = main.UIGraphDisplayManager(display_output=display_output, 
                                             display_size=display_size,
                                             layout_vbox=layout_select_vbox,
                                             export_vbox=export_vbox,
                                             cumulative_hbox=cumulative_hbox,
                                             loading_indicator=loading_graph,
                                             style_manager=style_manager)
    # Show import view
    full_import_vbox.layout.display = 'block'
        
run_button.on_click(on_run)
autostart_button.on_click(on_autorun)
import_menu_button.on_click(on_import)

# Import Box
ipydisplay.display(full_import_vbox)

In [ ]:
ipydisplay.display(main_stats_hbox)

In [ ]:
ipydisplay.display(ipydisplay.HTML('''
<script>
    code_show=true; 
    function code_toggle() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
    } 
     function observe_primary_btns() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
'''))